# Image caption generator

## Part 1 - seq2seq auto encoder

In [1]:
from fastai.text import *
import pandas as pd
from pathlib import Path

In [2]:
PATH = Path('data/')
PATH.mkdir(exist_ok=True)

In [3]:
seq_PATH = PATH/'seq2deq_model'
seq_PATH.mkdir(exist_ok=True)

In [4]:
captions_and_links_valid = pd.read_csv("Validation_GCC-1.1.0-Validation.tsv", sep="\t",header=None)
captions_and_links_train = pd.read_csv("Train_GCC-training.tsv", sep="\t",header=None)

In [5]:
len(captions_and_links_valid), len(captions_and_links_train)

(15840, 3318333)

In [6]:
captions_and_links_valid.head()

,0,1
0,author : a life in photography -- in pictures,https://i.pinimg.com/736x/66/01/6c/66016c3ba27...
1,an angler fishes river on a snowy day .,http://www.standard.net/image/2015/02/04/800x_...
2,photograph of the sign being repaired by brave...,http://indianapolis-photos.funcityfinder.com/f...
3,the player staring intently at a computer scre...,http://www.abc.net.au/news/image/9066492-3x2-7...
4,globes : the green 3d person carrying in hands...,https://www.featurepics.com/StockImage/2009031...


In [7]:
captions_valid = captions_and_links_valid[0].values
captions_train = captions_and_links_train[0].values

In [8]:
len(captions_train), len(captions_valid)

(3318333, 15840)

### Tokenization

In [33]:
tokenizer = Tokenizer()

In [34]:
tokens_captions_valid = tokenizer.process_all(captions_valid)

In [36]:
tokens_captions_train = tokenizer.process_all(captions_train)

**Let's make sure everything went right:**

In [40]:
print(captions_train[777], ' '.join(tokens_captions_train[777]))

people click into their skis . people click into their skis .


### Length of captions
#### Average length of captions

In [45]:
np.mean([len(c) for c in tokens_captions_valid])

10.415467171717172

In [46]:
np.mean([len(c) for c in tokens_captions_train])

10.322056888202601

#### Discard captions that are too long

In [50]:
tokens_captions_valid = np.array(tokens_captions_valid)[np.array([len(c) < 30 for c in tokens_captions_valid])]

In [52]:
np.max([len(c) for c in tokens_captions_valid])

29

In [53]:
tokens_captions_train = np.array(tokens_captions_train)[np.array([len(c) < 30 for c in tokens_captions_train])]

In [55]:
np.max([len(c) for c in tokens_captions_train])

29

In [65]:
pickle.dump((tokens_captions_valid), (seq_PATH/'tok_cap_v.pkl').open('wb'))

In [66]:
pickle.dump((tokens_captions_train), (seq_PATH/'tok_cap_t.pkl').open('wb'))

In [7]:
tokens_captions_valid = pickle.load((seq_PATH/'tok_cap_v.pkl').open('rb'))

In [8]:
tokens_captions_train = pickle.load((seq_PATH/'tok_cap_t.pkl').open('rb'))

In [9]:
print(len(tokens_captions_train), len(tokens_captions_valid))

3302516 15764


### Numericalization

In [10]:
vocab = Vocab.create(tokens_captions_train, max_vocab=50000, min_freq=2)

**Let's quickly test this:**

In [11]:
idx = 12575

In [12]:
test_caption = tokens_captions_valid[idx]
print(' '.join(w for w in test_caption))

actor attending the world premiere of crime fiction film


In [13]:
test_caption_num = vocab.numericalize(test_caption)
print(test_caption_num)

[31, 1841, 8, 108, 43, 11, 2274, 1018, 76]


In [14]:
print(vocab.textify(test_caption_num))

actor attending the world premiere of crime fiction film


In [15]:
tokens_captions_valid[0]

['author', ':', 'a', 'life', 'in', 'photography', '--', 'in', 'pictures']

In [16]:
def numericalize_tokens(tok):
    return np.array([vocab.numericalize(q) + [1] for q in tok])

In [17]:
captions_numericalized_valid = numericalize_tokens(tokens_captions_valid)

In [18]:
captions_numericalized_train = numericalize_tokens(tokens_captions_train)

Test it:

In [19]:
vocab.textify(captions_numericalized_train[1256339])

'a vector illustration of a motorcycle contrasted against a bicycle . xxpad'

### Word vectors

In [ ]:
#! pip install git+https://github.com/facebookresearch/fastText.git

In [50]:
import fastText as ft

In [51]:
en_vecs = ft.load_model(str(seq_PATH/'wiki.en.bin'))

In [110]:
vec_dict = {w : en_vecs.get_word_vector(w) for w in en_vecs.get_words()}

In [113]:
pickle.dump(vec_dict, open(seq_PATH/'vec_dict.pkl', 'wb'))

In [20]:
vec_dict = pickle.load(open(seq_PATH/'vec_dict.pkl','rb'))

**Let's take a look at the most frequent words from fastText:**

In [115]:
ft_words = en_vecs.get_words(include_freq=True)

In [117]:
ft_word_dict = {k:v for k,v in zip(*ft_words)}

In [118]:
ft_words = sorted(ft_word_dict.keys(), key=lambda x: ft_word_dict[x])

10 most frequent words:

In [123]:
ft_words[-10:]

[')', "'", 'and', 'in', '-', 'of', '</s>', 'the', '.', ',']

**Mean and stdv of the word vectors:**

In [17]:
vecs = np.stack(list(vec_dict.values()))

In [18]:
vecs.mean(), vecs.std()

(0.0075652334, 0.29283327)

### Dataset

In [21]:
def A(*a):
    """convert iterable object into numpy array"""
    return np.array(a[0]) if len(a)==1 else [np.array(o) for o in a]

In [156]:
class Caption2CaptionDataset(Dataset):
    def __init__(self, x):
        self.x = x
    def __getitem__(self, idx):
        return A(self.x[idx], self.x[idx])
    def __len__(self):
        return len(self.x)

In [157]:
trn_ds = Caption2CaptionDataset(captions_numericalized_train)

In [158]:
val_ds = Caption2CaptionDataset(captions_numericalized_valid)

In [159]:
len(trn_ds), len(val_ds)

(3302516, 15764)

In [160]:
vocab.textify(trn_ds[0][0])

'a very typical bus station xxpad'

In [161]:
bs = 125

In [162]:
val_sampler = SortSampler(captions_numericalized_valid, key=lambda x:len(captions_numericalized_valid[x]))

In [163]:
trn_sampler = SortishSampler(captions_numericalized_train, key=lambda x: len(captions_numericalized_train[x]), bs=bs)

Since we want to fully utilize the GPUs capabilities, we train in batches. The length of a minibatch tensor is set by the sequence length of the longest question in that batch. The other questions are padded. To save time and memory, we want to avoid very long and very short questions in one batch because that would mean lot's of padding. For the validation set we simply sort the questions. For training we use the `SortishSampler` which groups *longer* questions together and *shorter* questions together while preserving some randomness.

For language models it's better to pad before the start of the sequence because we need the final hidden state to predict the next token or for classification...

For sequence to sequence models it is better to pad after the end of the sequence.

The samplers return an index:

In [164]:
i = next(iter(trn_sampler))

In [165]:
vocab.textify(captions_numericalized_train[i])

'this tree , which fell only metres from a residence , was one of hundreds that suffered extensive damage at the hands of gusting winds on saturday afternoon . xxpad'

In [166]:
import pdb

In [167]:
def pad_collate_seq2seq(samples:BatchSamples, pad_idx:int=1, pad_first:bool=True, backwards:bool=False, transpose:bool=False) -> Tuple[LongTensor, LongTensor]:
    "Function that collect samples and adds padding. Flips token order if needed"
    samples = to_data(samples)
    max_len_inp = max([len(s[0]) for s in samples])
    max_len_out = max([len(s[1]) for s in samples])
    
    res_inp = torch.zeros(len(samples), max_len_inp).long() + pad_idx
    res_out = torch.zeros(len(samples), max_len_out).long() + pad_idx
    
    if backwards: pad_first = not pad_first
    for i,s in enumerate(samples):
        if pad_first: 
            res_inp[i,-len(s[0]):] = LongTensor(s[0])
            res_out[i,-len(s[1]):] = LongTensor(s[1])
        else:         
            res_inp[i,:len(s[0]):] = LongTensor(s[0])
            res_out[i,:len(s[1]):] = LongTensor(s[1])
    if backwards:
        res = res.flip(1)
    if transpose:
        res_inp.transpose_(0,1)
        res_out.transpose_(0,1)
    return res_inp, res_out

In [168]:
def jag_stack(self, b):
    if len(b[0].shape) not in (1,2): return np.stack(b)
    ml = max(len(o) for o in b)
    if min(len(o) for o in b)==ml: return np.stack(b)
    res = np.zeros((len(b), ml), dtype=b[0].dtype) + self.pad_idx
    for i,o in enumerate(b):
        if self.pre_pad: res[i, -len(o):] = o
        else:            res[i,  :len(o)] = o
    return res


In [169]:
my_pad_collate_func = partial(pad_collate_seq2seq, pad_first=False, transpose=True)

In [170]:
trn_dl = DataLoader(dataset=trn_ds, batch_size=bs, sampler=trn_sampler, collate_fn=my_pad_collate_func)

In [175]:
val_dl = DataLoader(dataset=val_ds, batch_size=bs, sampler=val_sampler, collate_fn=my_pad_collate_func)

Let's test the dataloader:

In [190]:
it = iter(val_dl)

In [191]:
its = [next(it) for _ in range(3)]

In [192]:
[(x.shape, y.shape) for x, y in its]

[(torch.Size([30, 125]), torch.Size([30, 125])),
 (torch.Size([27, 125]), torch.Size([27, 125])),
 (torch.Size([24, 125]), torch.Size([24, 125]))]

In [198]:
for x, y in its:
    print(vocab.textify(x[:,100]))
    print(vocab.textify(y[:,100]))
    print()

this is an old photo of my figure i had and i slightly edited it and now he looks like he 's from a dramatic anime xxpad xxpad xxpad xxpad
this is an old photo of my figure i had and i slightly edited it and now he looks like he 's from a dramatic anime xxpad xxpad xxpad xxpad

i have no idea how that american flag got in this picture of the construction , but i love that it is there ! xxpad xxpad xxpad
i have no idea how that american flag got in this picture of the construction , but i love that it is there ! xxpad xxpad xxpad

detail shot : the hands of a woman driving a car by the countryside , seen from the front passenger 's seat xxpad xxpad
detail shot : the hands of a woman driving a car by the countryside , seen from the front passenger 's seat xxpad xxpad

